In [1]:
import pandas as pd
from statsbombpy import sb
from mplsoccer import Sbopen

In [15]:
parser = Sbopen()
partido = parser.event(match_id=3869685)
eventos_partido = partido[0]

In [16]:
eventos_partido.columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession',
       'duration', 'match_id', 'type_id', 'type_name', 'possession_team_id',
       'possession_team_name', 'play_pattern_id', 'play_pattern_name',
       'team_id', 'team_name', 'tactics_formation', 'player_id', 'player_name',
       'position_id', 'position_name', 'pass_recipient_id',
       'pass_recipient_name', 'pass_length', 'pass_angle', 'pass_height_id',
       'pass_height_name', 'end_x', 'end_y', 'sub_type_id', 'sub_type_name',
       'body_part_id', 'body_part_name', 'x', 'y', 'pass_switch', 'outcome_id',
       'outcome_name', 'under_pressure', 'off_camera', 'counterpress',
       'block_deflection', 'pass_deflected', 'foul_won_defensive', 'out',
       'pass_cross', 'technique_id', 'technique_name', 'aerial_won',
       'pass_assisted_shot_id', 'pass_shot_assist', 'shot_statsbomb_xg',
       'end_z', 'shot_key_pass_id', 'goalkeeper_position_id',
       'goalkeeper_position_name', 'ball_recovery_

In [17]:
eventos_partido = eventos_partido[eventos_partido['possession_team_name'] == eventos_partido['team_name']]

In [20]:
eventos_partido.groupby('possession').sum()['shot_statsbomb_xg']

possession
1      0.0000
2      0.0000
3      0.0000
4      0.0000
5      0.0000
        ...  
251    0.7835
252    0.7835
253    0.7835
254    0.7835
255    0.7835
Name: shot_statsbomb_xg, Length: 255, dtype: float64

In [ ]:
#Importante la necesidad de contar bien las posesiones
#Cuando en el eventing el nombre del equipo cambia (pero considerar que puede haber acciones que aparezcan como duelos aereo perdido que deberian desestimarse)

#Considerar también que las posesiones no siempre cambian cuando un equipo pierde la pelota. Faltas, laterales, corners también deberian ser considerados cuando el equipo mismo tiene la pelota 
#para entender si cambia la posesión.


#Una forma de contar posesiones
eventos_partido['posesion'] = (eventos_partido['possession_team_name'] != eventos_partido['possession_team_name'].shift()).cumsum()

"""
Otra
df['lag_team'] = df['teamId'].shift(1)
    df['mismo equipo'] = np.where(df['teamId'] == df['lag_team'], True,False)
    df['Posesion'] = 0
    n=0
    for i in range(len(df)):
        if df['mismo equipo'].iloc[i] == True:
            df['Posesion'].iloc[i] = n
        else:
            df['Posesion'].iloc[i] = n+1
            n+=1 

"""

In [24]:
xg_values_possession = eventos_partido.groupby('possession', as_index=False).sum()[['possession','shot_statsbomb_xg']]
xg_values_possession = xg_values_possession[xg_values_possession['shot_statsbomb_xg'] != 0].rename(columns={'shot_statsbomb_xg': 'xGChain'})
xg_values_possession

,possession,xGChain
13,14,0.024542
17,18,0.062813
26,27,0.106801
31,32,0.783500
45,46,0.010578
51,52,0.303409
85,86,0.040216
106,107,0.059534
115,116,0.115560
130,131,0.096184


In [31]:
xGChain_df = eventos_partido.merge(xg_values_possession, on='possession', how='left')

In [37]:
xGChain_df = xGChain_df[xGChain_df['possession'] < 247]

In [38]:
xGChain_df[(xGChain_df['xGChain'] != 0)].drop_duplicates(subset=['player_name', 'possession']).groupby('player_name').sum()['xGChain'].sort_values(ascending=False)

player_name
Lionel Andrés Messi Cuccittini     2.563312
Kylian Mbappé Lottin               1.782554
Enzo Fernandez                     1.224668
Lautaro Javier Martínez            1.092528
Gonzalo Ariel Montiel              0.996280
Alexis Mac Allister                0.798650
Ángel Fabián Di María Hernández    0.698911
Nicolás Alejandro Tagliafico       0.691139
Leandro Daniel Paredes             0.592476
Julián Álvarez                     0.565079
Marcos Javier Acuña                0.527542
Nahuel Molina Lucero               0.496132
Randal Kolo Muani                  0.373698
Rodrigo Javier De Paul             0.312858
Hugo Lloris                        0.277514
Ibrahima Konaté                    0.277514
Kingsley Coman                     0.248827
Adrien Rabiot                      0.245682
Eduardo Camavinga                  0.227487
Nicolás Hernán Otamendi            0.190319
Dayotchanculle Upamecano           0.155492
Antoine Griezmann                  0.146828
Aurélien Djani Tchou

In [42]:
xGChain_df[(xGChain_df['xGChain'] != 0) & (xGChain_df['type_name'] != 'Shot') &  (xGChain_df['pass_shot_assist'] != True)].drop_duplicates(subset=['player_name', 'possession']).groupby('player_name').sum()['xGChain'].sort_values(ascending=False)

player_name
Lionel Andrés Messi Cuccittini     1.779812
Enzo Fernandez                     1.224668
Lautaro Javier Martínez            1.092528
Gonzalo Ariel Montiel              0.971495
Alexis Mac Allister                0.798650
Ángel Fabián Di María Hernández    0.698911
Nicolás Alejandro Tagliafico       0.691139
Leandro Daniel Paredes             0.592476
Julián Álvarez                     0.540537
Marcos Javier Acuña                0.527542
Nahuel Molina Lucero               0.496132
Randal Kolo Muani                  0.373698
Rodrigo Javier De Paul             0.312858
Hugo Lloris                        0.277514
Ibrahima Konaté                    0.277514
Kingsley Coman                     0.248827
Adrien Rabiot                      0.245682
Eduardo Camavinga                  0.227487
Kylian Mbappé Lottin               0.215554
Nicolás Hernán Otamendi            0.190319
Dayotchanculle Upamecano           0.155492
Aurélien Djani Tchouaméni          0.143979
Paulo Bruno Exequiel

In [43]:
xGChain_df[(xGChain_df['xGChain'] != 0) & (xGChain_df['type_name'] != 'Shot') & (xGChain_df['pass_shot_assist'] != True)]

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,...,foul_committed_card_name,ball_recovery_offensive,shot_one_on_one,foul_committed_advantage,foul_won_advantage,block_offensive,foul_committed_offensive,bad_behaviour_card_id,bad_behaviour_card_name,xGChain
0,0584ee21-e3dd-4d9f-95a0-5b5e84be25c3,1,1,00:00:00,0,0,1,0.000000,3869685,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6404a8e8-afaf-489d-b65e-173a237ffed5,4,1,00:00:00,0,0,1,0.000000,3869685,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f651a6c4-55e3-4e0f-a178-59414ba83d6a,5,1,00:00:00.578000,0,0,2,0.975702,3869685,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97b5dc82-547a-4f93-a632-a2a8daf5ac98,6,1,00:00:01.554000,0,1,2,NaN,3869685,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9707e590-1d55-4d25-a50f-475019356152,7,1,00:00:01.554000,0,1,2,1.170175,3869685,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3531,7a530d34-8772-465d-957e-e26a59aec795,4350,4,00:17:52.805000,122,52,246,2.013526,3869685,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10655
3532,494f058f-f207-4eaf-8572-fa8231480ea1,4351,4,00:17:54.818000,122,54,246,2.847939,3869685,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10655
3533,cbfdc80d-9535-437d-acd3-b0112a420cc9,4353,4,00:17:57.666000,122,57,246,NaN,3869685,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10655
3534,7b17edc2-ad6e-4aa5-b978-ec44fd7c31bc,4354,4,00:17:57.666000,122,57,246,1.110587,3869685,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10655
